# ETL Pipeline Preparation
Follow the instructions below to help you create your ETL pipeline.
### 1. Import libraries and load datasets.
- Import Python libraries
- Load `messages.csv` into a dataframe and inspect the first few lines.
- Load `categories.csv` into a dataframe and inspect the first few lines.

In [1]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
!pip install langdetect
from langdetect import detect



You are using pip version 9.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
# load messages dataset
messages = pd.read_csv("messages.csv") 
messages.head()

,id,message,original,genre
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct


In [3]:
messages.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26248 entries, 0 to 26247
Data columns (total 4 columns):
id          26248 non-null int64
message     26248 non-null object
original    10184 non-null object
genre       26248 non-null object
dtypes: int64(1), object(3)
memory usage: 820.3+ KB


In [4]:
messages.genre.unique()

array(['direct', 'social', 'news'], dtype=object)

In [5]:
# load categories dataset
categories = pd.read_csv("categories.csv")
categories.head()

,id,categories
0,2,related-1;request-0;offer-0;aid_related-0;medi...
1,7,related-1;request-0;offer-0;aid_related-1;medi...
2,8,related-1;request-0;offer-0;aid_related-0;medi...
3,9,related-1;request-1;offer-0;aid_related-1;medi...
4,12,related-1;request-0;offer-0;aid_related-0;medi...


In [6]:
categories.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26248 entries, 0 to 26247
Data columns (total 2 columns):
id            26248 non-null int64
categories    26248 non-null object
dtypes: int64(1), object(1)
memory usage: 410.2+ KB


In [7]:
# merge datasets
df = messages.merge(categories, how='inner',on=['id'])
df.head()

,id,message,original,genre,categories
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,related-1;request-0;offer-0;aid_related-0;medi...
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,related-1;request-0;offer-0;aid_related-1;medi...
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,related-1;request-0;offer-0;aid_related-0;medi...
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,related-1;request-1;offer-0;aid_related-1;medi...
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,related-1;request-0;offer-0;aid_related-0;medi...


### 3. Split `categories` into separate category columns.
- Split the values in the `categories` column on the `;` character so that each value becomes a separate column. You'll find [this method](https://pandas.pydata.org/pandas-docs/version/0.23/generated/pandas.Series.str.split.html) very helpful! Make sure to set `expand=True`.
- Use the first row of categories dataframe to create column names for the categories data.
- Rename columns of `categories` with new column names.

In [8]:
# create a dataframe of the 36 individual category columns
categories = df.categories.str.split(';',expand=True)
categories.head()

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
0,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
1,related-1,request-0,offer-0,aid_related-1,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-1,floods-0,storm-1,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
2,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
3,related-1,request-1,offer-0,aid_related-1,medical_help-0,medical_products-1,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
4,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0


In [9]:
# select the first row of the categories dataframe
row = categories.loc[0]

# use this row to extract a list of new column names for categories.
# one way is to apply a lambda function that takes everything 
# up to the second to last character of each string with slicing
category_colnames = row.apply(lambda x:x.split("-")[0])
print(category_colnames)

0                    related
1                    request
2                      offer
3                aid_related
4               medical_help
5           medical_products
6          search_and_rescue
7                   security
8                   military
9                child_alone
10                     water
11                      food
12                   shelter
13                  clothing
14                     money
15            missing_people
16                  refugees
17                     death
18                 other_aid
19    infrastructure_related
20                 transport
21                 buildings
22               electricity
23                     tools
24                 hospitals
25                     shops
26               aid_centers
27      other_infrastructure
28           weather_related
29                    floods
30                     storm
31                      fire
32                earthquake
33                      cold
34            

In [10]:
# rename the columns of `categories`
categories.columns = category_colnames
categories.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
1,related-1,request-0,offer-0,aid_related-1,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-1,floods-0,storm-1,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
2,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
3,related-1,request-1,offer-0,aid_related-1,medical_help-0,medical_products-1,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
4,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,child_alone-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0


### 4. Convert category values to just numbers 0 or 1.
- Iterate through the category columns in df to keep only the last character of each string (the 1 or 0). For example, `related-0` becomes `0`, `related-1` becomes `1`. Convert the string to a numeric value.
- You can perform [normal string actions on Pandas Series](https://pandas.pydata.org/pandas-docs/stable/text.html#indexing-with-str), like indexing, by including `.str` after the Series. You may need to first convert the Series to be of type string, which you can do with `astype(str)`.

In [11]:
for column in categories:
    # set each value to be the last character of the string
    categories[column] = categories[column].apply(lambda x:x.split("-")[1])
    
    # convert column from string to numeric
    categories[column] = pd.to_numeric(categories[column])
categories.head()

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
for column in categories:
    print(column, " ",categories[column].unique())


related   [1 0 2]
request   [0 1]
offer   [0 1]
aid_related   [0 1]
medical_help   [0 1]
medical_products   [0 1]
search_and_rescue   [0 1]
security   [0 1]
military   [0 1]
child_alone   [0]
water   [0 1]
food   [0 1]
shelter   [0 1]
clothing   [0 1]
money   [0 1]
missing_people   [0 1]
refugees   [0 1]
death   [0 1]
other_aid   [0 1]
infrastructure_related   [0 1]
transport   [0 1]
buildings   [0 1]
electricity   [0 1]
tools   [0 1]
hospitals   [0 1]
shops   [0 1]
aid_centers   [0 1]
other_infrastructure   [0 1]
weather_related   [0 1]
floods   [0 1]
storm   [0 1]
fire   [0 1]
earthquake   [0 1]
cold   [0 1]
other_weather   [0 1]
direct_report   [0 1]


## Observations
- Feature 'related' has three unique values(0,1,2). What does 2 mean, is investigated below.
- Feature 'child_alone' has only one unique value which is 0.

In [13]:
related_2_subset = categories[categories.related==2]
related_1_subset = categories[categories.related==1]
related_0_subset = categories[categories.related==0]

In [14]:
for column in related_2_subset:
    print(column, " ",related_2_subset[column].unique())

related   [2]
request   [0]
offer   [0]
aid_related   [0]
medical_help   [0]
medical_products   [0]
search_and_rescue   [0]
security   [0]
military   [0]
child_alone   [0]
water   [0]
food   [0]
shelter   [0]
clothing   [0]
money   [0]
missing_people   [0]
refugees   [0]
death   [0]
other_aid   [0]
infrastructure_related   [0]
transport   [0]
buildings   [0]
electricity   [0]
tools   [0]
hospitals   [0]
shops   [0]
aid_centers   [0]
other_infrastructure   [0]
weather_related   [0]
floods   [0]
storm   [0]
fire   [0]
earthquake   [0]
cold   [0]
other_weather   [0]
direct_report   [0]


In [15]:
for column in related_0_subset:
    print(column, " ",related_0_subset[column].unique())

related   [0]
request   [0]
offer   [0]
aid_related   [0]
medical_help   [0]
medical_products   [0]
search_and_rescue   [0]
security   [0]
military   [0]
child_alone   [0]
water   [0]
food   [0]
shelter   [0]
clothing   [0]
money   [0]
missing_people   [0]
refugees   [0]
death   [0]
other_aid   [0]
infrastructure_related   [0]
transport   [0]
buildings   [0]
electricity   [0]
tools   [0]
hospitals   [0]
shops   [0]
aid_centers   [0]
other_infrastructure   [0]
weather_related   [0]
floods   [0]
storm   [0]
fire   [0]
earthquake   [0]
cold   [0]
other_weather   [0]
direct_report   [0]


In [16]:
for column in related_1_subset:
    print(column, " ",related_1_subset[column].unique())

related   [1]
request   [0 1]
offer   [0 1]
aid_related   [0 1]
medical_help   [0 1]
medical_products   [0 1]
search_and_rescue   [0 1]
security   [0 1]
military   [0 1]
child_alone   [0]
water   [0 1]
food   [0 1]
shelter   [0 1]
clothing   [0 1]
money   [0 1]
missing_people   [0 1]
refugees   [0 1]
death   [0 1]
other_aid   [0 1]
infrastructure_related   [0 1]
transport   [0 1]
buildings   [0 1]
electricity   [0 1]
tools   [0 1]
hospitals   [0 1]
shops   [0 1]
aid_centers   [0 1]
other_infrastructure   [0 1]
weather_related   [0 1]
floods   [0 1]
storm   [0 1]
fire   [0 1]
earthquake   [0 1]
cold   [0 1]
other_weather   [0 1]
direct_report   [0 1]


## Observations
Feature 'related' with value 0 means that the message is not related to disaster. It can be seen that for related:0 all the other fetures, have values 0. Intrestingly, it is also true for related:2. As these features are binary features and related:2 does not provide any additional/different information, I am making the assumption that related:2 is actually relared:0.

In [17]:
categories.related[categories.related==2] = 0

In [18]:
categories.related.unique()

array([1, 0])

In [19]:
def child_alone_messages(message):
    if("child" in message.lower() or "alone" in message.lower()):
        print(message)
    else:
        return
    

In [20]:
df.message.apply(child_alone_messages)

My house is destroyed in Carrefour. My house is destroyed in Port-au-Prince. The money that the government gave me is lost. My children die, but I am not hurt. Please if you can help me protect the house. ...
I would like to leave this country and at my parents house in the USA. We are alone in Haiti. This is my number
S. O. S. Come to our rescue regarding water, food and medical attention and perticularly tents for us to sleep well with the small children
they closed the bathroom and shower at the TNH, so we can no longer do our physical needs. .. . there is some children that poop on themselve because. .. Note : Message cutoff, need to look at other. .. . Sebastien Cournoyer
Please I am alone and I'd like to know if there is somebody out there who would like to adopt an adolescent please adopt me.
Please help us so that hunger doesn't kill the children
Things are so difficult for me and my child. Thanks
Mother has 4 children and 3 other children from a sister who died and, 1 child fr

0        None
1        None
2        None
3        None
4        None
5        None
6        None
7        None
8        None
9        None
10       None
11       None
12       None
13       None
14       None
15       None
16       None
17       None
18       None
19       None
20       None
21       None
22       None
23       None
24       None
25       None
26       None
27       None
28       None
29       None
         ... 
26356    None
26357    None
26358    None
26359    None
26360    None
26361    None
26362    None
26363    None
26364    None
26365    None
26366    None
26367    None
26368    None
26369    None
26370    None
26371    None
26372    None
26373    None
26374    None
26375    None
26376    None
26377    None
26378    None
26379    None
26380    None
26381    None
26382    None
26383    None
26384    None
26385    None
Name: message, Length: 26386, dtype: object

## Observation
- Feature 'child_alone' had only 0 as the unique value. It means that in the dataset, no message is there about a lone child. Here, we need to question the logic behind this feature. Are we expecting a child to write a message or tweet? What is the definition of a child. A lot of lony childrenmight not have access to a phone.   
- This feature might have value 1 only when the genre is news. For now, I am leaving it as it is as more information is needed. 
- I was checking the individual messages here and I have tears in my eyes. These disasters are terrible for the families. 

### 5. Replace `categories` column in `df` with new category columns.
- Drop the categories column from the df dataframe since it is no longer needed.
- Concatenate df and categories data frames.

In [21]:
df.head()

,id,message,original,genre,categories
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,related-1;request-0;offer-0;aid_related-0;medi...
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,related-1;request-0;offer-0;aid_related-1;medi...
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,related-1;request-0;offer-0;aid_related-0;medi...
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,related-1;request-1;offer-0;aid_related-1;medi...
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,related-1;request-0;offer-0;aid_related-0;medi...


In [22]:
# drop the original categories column from `df`

df.drop(['categories'], axis=1, inplace=True)
df.head()

,id,message,original,genre
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct


In [23]:
# concatenate the original dataframe with the new `categories` dataframe
df = pd.concat([df, categories], axis=1)
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 6. Remove duplicates.
- Check how many duplicates are in this dataset.
- Drop the duplicates.
- Confirm duplicates were removed.

In [24]:
# check number of duplicates
df.id.duplicated().sum()

206

In [25]:
# drop duplicates
df = df[df.id.duplicated()==False]

In [26]:
# check number of duplicates
df.id.duplicated().sum()

0

### 6. Remove Non English Messages.
- Check the language of the message and the original column. 
- Count the non-Englih message rows
- For rows in which language for Original Column is English and message column is not english,Copy the text from original column to message column
- Remove all the rows with non-English text in message column

In [27]:
def lang_detect(txt):
    try:
        return detect(txt)
    except:
        return None

In [28]:
df['message_lang'] = df.message.apply(lambda x: lang_detect(x))

In [29]:
df.message_lang.value_counts()

en    25793
fr       54
de       51
pt       33
id       32
af       31
nl       27
so       24
es       19
ca       16
it       15
da       12
vi       12
tl        8
et        8
cy        6
no        6
sq        5
sw        4
sv        4
sl        3
pl        3
fi        3
hr        2
tr        2
sk        1
ro        1
Name: message_lang, dtype: int64

In [30]:
# Number of rows with message in non-English
df.shape[0] - df[df.message_lang=='en'].shape[0]

387

In [31]:
df['original_lang'] = df.original.apply(lambda x: lang_detect(x))

In [32]:
df[df.original_lang.isin(['en']) & ~df.message_lang.isin(['en'])].shape[0]

32

In [33]:
df.message[df.original_lang.isin(['en']) & ~df.message_lang.isin(['en'])] = df.original
df.message_lang[df.original_lang.isin(['en']) & ~df.message_lang.isin(['en'])] = "en"

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [34]:
df.shape[0] - df[df.message_lang=='en'].shape[0]

355

In [35]:
df_not_eng = df[df.message_lang!="en"]

In [36]:
df_not_eng.related.value_counts()

1    188
0    167
Name: related, dtype: int64

In [37]:
df_eng = df[df.message_lang=="en"]
df_eng.shape[0]

25825

In [39]:
df_eng = df[df.message_lang.notnull()]
df_eng = df_eng.drop(['message_lang','original_lang'], axis=1)

In [40]:
df_eng.columns

Index(['id', 'message', 'original', 'genre', 'related', 'request', 'offer',
       'aid_related', 'medical_help', 'medical_products', 'search_and_rescue',
       'security', 'military', 'child_alone', 'water', 'food', 'shelter',
       'clothing', 'money', 'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')

### 7. Save the clean dataset into an sqlite database.
You can do this with pandas [`to_sql` method](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_sql.html) combined with the SQLalchemy library.

In [41]:
engine = create_engine('sqlite:///DisasterResponse.db')
df_eng.to_sql('Messages', engine, if_exists='replace',index=False)

### 8. Use this notebook to complete `etl_pipeline.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database based on new datasets specified by the user. Alternatively, you can complete `etl_pipeline.py` in the classroom on the `Project Workspace IDE` coming later.